In [78]:
import content_extractor

import configparser

from importlib import reload

# set the max columns to none
import pandas as pd
pd.set_option('display.max_columns', None)
# set max_colwidth to 3000
pd.set_option('display.max_colwidth', 100)

In [127]:
reload(content_extractor)

config = configparser.ConfigParser()

config_file_path = 'config/test.ini'
config.read(config_file_path)

# Get configuration options
input_filename = config.get('General', 'input_filename')
output_filename = config.get('General', 'output_filename')
output_filename = None if output_filename == "None" else output_filename
mode = config.get('General', 'mode')
num_processes = config.getint('General', 'num_processes')
url_col_name = config.get('General', 'url_col_name')
pub_date_col_name = config.get('General', 'pub_date_col_name')

if mode in {'openai', 'all'}:
    openai_model = config.get('General', 'openai_model')
    openai_temp = config.getfloat('General', 'openai_temp')
    openai_max_tokens = config.getint('General', 'openai_max_tokens') 
    
    # Initialize ContentExtractor
    extractor = content_extractor.ContentExtractor(openai_model, openai_temp, openai_max_tokens)

elif mode == 'extractor': extractor = content_extractor.ContentExtractor()
    

In [ ]:
# Read data
data_df = extractor.read_data(input_filename, url_col_name=url_col_name, pub_date_col_name=pub_date_col_name)
data_df

In [142]:
url_content_df = extractor.extract_content(data_df, num_processes=num_processes, out_fn=output_filename)

INFO:root:Saving results ...
INFO:root:Saved.


In [94]:
url_content_df = data_df
url_content_df = extractor.filter_scraped_data(url_content_df)
url_content_df

,PublishedDate,Language,Alert_definition,PT,Title,Content,Id,URL,Summary,New_Content,Is_Article
0,2023-10-05T07:50:39Z,en,flood,all,Flash <b>floods</b> kill at least 14 in northeastern India and leave more than 100 missing,NEW DELHI (AP) — Rescue workers were searching for more than 100 people on Thursday after flash ...,"tag:google.com,2013:googlealerts/feed:7377215587900989542",https://nationalpost.com/pmn/news-pmn/flash-floods-kill-at-least-14-in-northeastern-india-and-le...,Article content It was the latest deadly flood to hit northeast India in a year of unusually hea...,"This advertisement has not loaded yet, but your article continues below. Icy floodwaters kill at...",1
1,2023-10-05T07:44:07Z,en,flood,all,"8 dead, 22 armymen among 69 missing as flash <b>flood</b> wreaks havoc in Sikkim; Modi dials CM","Besides the 22 army personnel, 47 civilians were also missing, while 166 people have so far been...","tag:google.com,2013:googlealerts/feed:1218474220935360231",https://m.timesofindia.com/city/kolkata/8-dead-22-armymen-among-69-missing-as-flash-flood-wreaks...,"""Spoke to Sikkim CM Shri PSTamangGolay and took stock of the situation in the wake of the unfort...",Sikkim flash floods: 23 army men missing after cloud burst in Lachen Valley NEW DELHI/GANGTOK: A...,1
2,2023-10-05T06:57:51Z,en,flood,all,"Tokyo Will Get a New Immersive Shopping, Dining Spot as Tourists <b>Flood</b> to Japan","(Bloomberg) -- A new immersive dining, shopping and entertainment attraction will open in Tokyo ...","tag:google.com,2013:googlealerts/feed:11568202952123180859",https://www.bnnbloomberg.ca/tokyo-will-get-a-new-immersive-shopping-dining-spot-as-tourists-floo...,"Tokyo Will Get a New Immersive Shopping, Dining Spot as Tourists Flood to Japan(Bloomberg) -- A ...","Tokyo Will Get a New Immersive Shopping, Dining Spot as Tourists Flood to Japan (Bloomberg) -- A...",1
3,2023-10-05T06:15:39Z,en,flood,all,New York&#39;s <b>floods</b> weren&#39;t a one-off — here&#39;s how the city is preparing for th...,New York flash <b>flooding</b> last week confirmed that climate change is outpacing infrastructu...,"tag:google.com,2013:googlealerts/feed:418994847420380038",https://www.cnbc.com/2023/10/04/new-york-city-flood-infrastructure-climate-chief.html,The same idea could be applied to create cloudburst gardens. Source: New York City Department of...,"A general view of a flooded street in Williamsburg, New York, United States on September 29, 202...",1
4,2023-10-04T15:31:03Z,fr,inondation,all,Inde | Au moins cinq morts et 23 soldats disparus après des crues subites dans l&#39;Himalaya,"« Des précipitations intenses ont conduit à cette situation catastrophique au Sikkim, où la plui...","tag:google.com,2013:googlealerts/feed:4038413060040291403",https://www.lapresse.ca/international/asie-et-oceanie/2023-10-04/inde/au-moins-cinq-morts-et-23-...,"(Guwahati) Au moins cinq personnes ont été tuées et des dizaines portées disparues, dont 23 sold...","(Guwahati) Au moins cinq personnes ont été tuées et des dizaines portées disparues, dont 23 sold...",1
5,2023-10-04T11:57:43Z,fr,inondation,all,Convoitise et inquiétudes en Libye concernant la reconstruction - Le Devoir,Les administrations rivales en Libye multiplient les initiatives pour s&#39;approprier une part ...,"tag:google.com,2013:googlealerts/feed:3481619434203993670",https://www.ledevoir.com/monde/afrique/799286/inondations-a-derna-convoitise-inquietudes-libye-c...,Les administrations rivales en Libye multiplient les initiatives pour sapproprier une part du gâ...,Les administrations rivales en Libye multiplient les initiatives pour sapproprier une part du gâ...,1
6,2023-10-05T07:50:39Z,en,NaN,NaN,NaN,NaN,NaN,https://www.cbc.ca/news/canada/newfoundland-labrador/post-tropical-storm-nicole-port-aux-basques...,Port aux Basques experienced minor flooding Saturday as the remnants of post-tropical storm Nico...,Port aux Basques experienced minor flooding Saturday as the remnants of post-tropical storm Nico...,1
7,2

In [128]:
# Run example om a sigle URL with OpenAI

tmp_df = url_content_df.iloc[8]
openai_content_0 = extractor.extract_single_event_chatopenai(tmp_df['New_Content'], tmp_df['URL'], tmp_df['Language'], tmp_df['PublishedDate'])
# content_df = extractor.transform_openai_response_to_df(openai_content_0)
# content_df

INFO:content_extractor:OpenAI is extracting information from https://www.winnipegfreepress.com/our-communities/west/overland-flooding-soaks-municipalities-576463972.html


In [129]:
openai_content_0

,is_happened,flood_cause_en,date,location,death,evacuation,country,link,published_date
0,Yes,Heavy snowfall and rainstorm,2022-04,"R.M. of Headingley, R.M. of Rosser, R.M. of MacDonald, R.M. of Cartier",No,No,Canada,https://www.winnipegfreepress.com/our-communities/west/overland-flooding-soaks-municipalities-57...,2023-10-05T07:50:39Z


In [74]:
extractor.transform_openai_response_to_df(openai_content_0)

,is_happened,flood_cause_en,date,location,death,evacuation,country
0,Oui,Rupture de deux barrages en amont de Derna,Inconnu,Derna,Oui,Oui,Libye


In [65]:
# Run example om a sigle URL with OpenAI

tmp_df = url_content_df.iloc[2]
openai_content = extractor.extract_single_event_chatopenai(tmp_df['New_Content'], tmp_df['URL'], tmp_df['Language'], tmp_df['PublishedDate'])

INFO:content_extractor:OpenAI is extracting information from https://www.bnnbloomberg.ca/tokyo-will-get-a-new-immersive-shopping-dining-spot-as-tourists-flood-to-japan-1.1980535


In [66]:
openai_content

'{\n  "1": "No",\n  "2": "NA",\n  "3": "NA",\n  "4": "NA",\n  "5": "NA",\n  "6": "NA",\n  "7": "NA"\n}'

In [67]:
extractor.transform_openai_response_to_df(openai_content)

,is_happened,flood_cause_en,date,location,death,evacuation,country
0,No,NA,NA,NA,NA,NA,NA


In [89]:
# Run example om a sigle URL with OpenAI

tmp_df = url_content_df.iloc[0]
openai_content_3 = extractor.extract_single_event_chatopenai(tmp_df['New_Content'], tmp_df['URL'], tmp_df['Language'], tmp_df['PublishedDate'])

INFO:content_extractor:OpenAI is extracting information from https://nationalpost.com/pmn/news-pmn/flash-floods-kill-at-least-14-in-northeastern-india-and-leave-more-than-100-missing


In [90]:
openai_content_3

'{\n  "1": "Yes",\n  "2": "Overflow of a glacial lake",\n  "3": "Unknown",\n  "4": "Sikkim State, India",\n  "5": "Yes",\n  "6": "Yes",\n  "7": "India"\n}'

In [86]:
extractor.transform_openai_response_to_df(openai_content_3)

,is_happened,flood_cause_en,date,location,death,evacuation,country
0,Yes,The flood was caused by the overflow of a glacial lake and the subsequent breach of a hydroelect...,Unknown,"The flood affected towns in India's Himalayan northeast, including Dikchu and Rangpo in the Tees...",Yes,Yes,India


In [102]:
tmp_df = url_content_df.iloc[7]
openai_model = "gpt-3.5-turbo-1106"
openai_content_7 = extractor.extract_single_event_chatopenai(tmp_df['New_Content'], tmp_df['URL'], tmp_df['Language'], tmp_df['PublishedDate'])

INFO:content_extractor:OpenAI is extracting information from https://www.saltwire.com/atlantic-canada/news/post-tropical-storm-earl-leaves-damaged-st-johns-roads-and-flooded-properties-in-its-wake-100772111/#:~:text=Post%2Dtropical%20storm%20Earl%20dumped,it%20was%20impassable%20Monday%20morning.


In [104]:
extractor.transform_openai_response_to_df(openai_content_7)

,is_happened,flood_cause_en,date,location,death,evacuation,country
0,Yes,Post-tropical storm Earl and heavy precipitation,2022-09,"St. John's, Mount Pearl, and surrounding areas",No,Yes,Canada


In [123]:
tmp_df = url_content_df.iloc[10]
openai_model = "gpt-3.5-turbo-1106"
openai_content_10 = extractor.extract_single_event_chatopenai(tmp_df['New_Content'], tmp_df['URL'], tmp_df['Language'], tmp_df['PublishedDate'])

INFO:content_extractor:OpenAI is extracting information from https://winnipeg.ctvnews.ca/overland-flooding-sparks-state-of-emergency-in-r-m-of-dufferin-1.5883827


In [124]:
openai_content_10

'{\n"1": "Yes",\n"2": "Excessive rain and river overflow",\n"3": "Unknown",\n"4": ["R.M. of Dufferin", "Town of Carman", "Town of Winnipeg Beach", "Town of Arborg", "R.M. of Gimli", "Long Plain First Nation", "City of Morden"],\n"5": "Unknown",\n"6": "Yes",\n"7": "Canada"\n}'

In [125]:
extractor.transform_openai_response_to_df(openai_content_10)

,is_happened,flood_cause_en,date,location,death,evacuation,country
0,Yes,Excessive rain and river overflow,Unknown,"[R.M. of Dufferin, Town of Carman, Town of Winnipeg Beach, Town of Arborg, R.M. of Gimli, Long P...",Unknown,Yes,Canada


In [234]:
import json
 # Replace newline characters and leading/trailing spaces
openai_content_31 = openai_content_3.replace('[', '').replace(']', '').replace('\n ', '')
# Remove any non-printable characters
openai_content_31 = ''.join(char for char in openai_content_31 if char.isprintable())
# openai_content_31[410:414]
openai_content_dict = json.loads(openai_content_31)


In [223]:
print(openai_content_31)

{  "1. Did a flood event occur?": "Yes",  "2. If a flood event occurred, what caused the flood event?": "Overflowing of a glacial lake",  "3. If a flood event occurred, when did it happen?": "Unknown",  "4. If a flood event occurred, where did it happen?": ["Sikkim State, India"],  "5. If a flood event occurred, how many casualties were there?": 31,  "6. If a flood event occurred, how many people were evacuated?": 2,000,  "7. If the locations of the flood-affected areas are known, which country are they in?": "India"}


In [217]:
json_str = '{"1. Did a flood event occur?": "Yes", "2. If a flood event occurred, what caused the flood event?": "Overflowing of a glacial lake","3. If a flood event occurred, when did it happen?": "Unknown","4. If a flood event occurred, where did it happen?": "Sikkim State, India","5. If a flood event occurred, how many casualties were there?": 31,  "6. If a flood event occurred, how many people were evacuated?": 2000,"7. If the locations of the flood-affected areas are known, which country are they in?": "India"}'
 
json.loads(json_str)

{'1. Did a flood event occur?': 'Yes',
 '2. If a flood event occurred, what caused the flood event?': 'Overflowing of a glacial lake',
 '3. If a flood event occurred, when did it happen?': 'Unknown',
 '4. If a flood event occurred, where did it happen?': 'Sikkim State, India',
 '5. If a flood event occurred, how many casualties were there?': 31,
 '6. If a flood event occurred, how many people were evacuated?': 2000,
 '7. If the locations of the flood-affected areas are known, which country are they in?': 'India'}

In [211]:
openai_content_31

'{ "1. Did a flood event occur?": "Yes", "2. If a flood event occurred, what caused the flood event?": "Overflowing of a glacial lake", "3. If a flood event occurred, when did it happen?": "Unknown", "4. If a flood event occurred, where did it happen?": "Sikkim State, India", "5. If a flood event occurred, how many casualties were there?": 31, "6. If a flood event occurred, how many people were evacuated?": 2,000, "7. If the locations of the flood-affected areas are known, which country are they in?": "India"}'

In [167]:
openai_content_df = pd.DataFrame([openai_content_dict])
openai_content_df

,1. Did a flood event occur?,"2. If a flood event occurred, what caused the flood event?","3. If a flood event occurred, when did it happen?","4. If a flood event occurred, where did it happen?","5. If a flood event occurred, how many casualties were there?","6. If a flood event occurred, how many people were evacuated?","7. If the locations of the flood-affected areas are known, which country are they in?"
0,Yes,Overflow of a glacial lake,Unknown,"Sikkim State, India",At least 31,"More than 2,000",India


In [155]:
content_df = extractor.transform_openai_response_to_df(openai_content_3)

content_df

ERROR:content_extractor:An error occurred during transformation: Expecting property name enclosed in double quotes: line 1 column 407 (char 406)


""


In [57]:
tmp_df = url_content_df.iloc[2]
openai_content_2 = extractor.extract_single_event_chatopenai(tmp_df['New_Content'], tmp_df['URL'], openai_model=openai_model, openai_temp=openai_temp, openai_max_tokens=openai_max_tokens)

INFO:content_extractor:Extracting information from https://www.winnipegfreepress.com/our-communities/west/overland-flooding-soaks-municipalities-576463972.html
INFO:content_extractor:Transforming information from https://www.winnipegfreepress.com/our-communities/west/overland-flooding-soaks-municipalities-576463972.html


In [58]:
content_df = extractor.transform_openai_response_to_df(openai_content_2)

content_df

,is_happened,flood_cause_en,date,location,death,evacuation,country
0,Yes,Heavy snowfall followed by rainstorm,2022-05,"Headingley (Dodds Road), RM of Headingley, RM of Rosser, RM of MacDonald, RM of Cartier",Unknown,Unknown,Canada
